# Part I. ETL Pipeline for Pre-Processing the Files

## FOLLOWING CODE IS FOR PRE-PROCESSING THE FILES

#### Import Necessary Python Packages 

In [64]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Create list of filepaths to process event csv data files

In [65]:
# Print current working directory 
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [66]:
# Create empty list to store the list of rows created from each file
full_data_rows_list = []
    
# Read every file within filepath
for f in file_path_list:

# Read .csv file
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # Extract data from each row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [67]:
# Check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("{} rows in event_datafile_new.csv".format(sum(1 for line in f)))

6821 rows in event_datafile_new.csv


In [68]:
df = pd.read_csv("event_datafile_new.csv")
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Connect to Cassandra and set Keyspace

#### Creating a Cluster

In [69]:
# Connect to a Cassandra instance on local machine

from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [70]:
# Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify 
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [71]:
# Set KEYSPACE to the keyspace above
try:
    session.set_keyspace('sparkify')
    
except Exception as e:
    print(e)

## Questions we need to be able to query for

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Create Tables in Apache Cassandra

In [72]:
## CREATE TABLES ##################################################
###################################################################
'''
Here I modeled the queries I would need to use to pull the relevant 
data for the questions asked above. 

Query 1 - For the first question I have created the song_library 
table. Since it is required to pull the information related to the 
songs from a specified 'Session ID' and number of 'Items in Session' 
I created a Partion Key composed of the 'sessionid', 'iteminsession'

Query 2 - For the second question I have created the artist_library 
table. Since it is required to pull the information related to the 
artists from a specified 'User ID' and 'Session ID' I created a 
Partion Key composed of the 'userid' and 'sessionid' with a 
clustering column based on the number of items in their session.

Query 3 - For the third question I have created the playback_library 
table. Here I created the table using a partion key with 'song_name' 
and 'userid'. I used the 'song_name' since we want to know everyone 
who listened to "All Hands Against His Own", and I used 'userid' 
since we wanted to make the query more efficent by allowing 
Cassandra to search on unique users instead of every copy that exists 
for each user.

---

Above each of the CREATE statements, I have included the query that 
we would need to answer the questions above.

'''

#### CREATE TABLE QUERIES #########################################
## QUERY 1 ####
#query1 = "select artist, song_name, song_length from song_library WHERE sessionId = 338 AND itemInSession = 4"
query1 = "CREATE TABLE IF NOT EXISTS song_library"
query1 = query1 + "(sessionid int, iteminsession int, artist varchar, song_name varchar, song_length float, PRIMARY KEY (sessionid, iteminsession))"
    
## QUERY 2 ####
#query2 = "select artist, song, firstname, lastname from song_library WHERE userid = 10 AND sessionid = 182"
query2 = "CREATE TABLE IF NOT EXISTS artist_library"
query2 = query2 + "(userid int, sessionid int, iteminsession int, artist varchar, song_name varchar, firstName varchar, lastname varchar, PRIMARY KEY ((userid, sessionid), iteminsession))"

## QUERY 3 ####
#query3 = "select firstname, lastname from playback_library WHERE song_name = 'All Hands Against His Own'"
query3 = "CREATE TABLE IF NOT EXISTS playback_library"
query3 = query3 + "(song_name varchar, userid int, firstName varchar, lastname varchar, PRIMARY KEY (song_name, userid))"


#### TRY CREATE QUERIES ############################################
try:
    session.execute(query1)
except Exception as e:
    print("Query 1: {}".format(e))

try:
    session.execute(query2)
except Exception as e:
    print("Query 2: {}".format(e))

try:
    session.execute(query3)
except Exception as e:
    print("Query 3: {}".format(e))
    

## Insert Data into Each Table

In [73]:
## POPULATE TABLES ##########################################
#############################################################

# Set file to read from
file = 'event_datafile_new.csv'

## INSERT QUERY 1 ####
iquery1 = "INSERT INTO song_library (sessionid, iteminsession, artist, song_name, song_length)"
iquery1 = iquery1 + " VALUES (%s, %s, %s, %s, %s)"

## INSERT QUERY 2 ####
iquery2 = "INSERT INTO artist_library (userid, sessionid, iteminsession, artist, song_name, firstName, lastname)"
iquery2 = iquery2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

## INSERT QUERY 3 ####
iquery3 = "INSERT INTO playback_library (song_name, userid, firstName, lastname)"
iquery3 = iquery3 + " VALUES (%s, %s, %s, %s)"

# Open file to read from 
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    # Read each line from file
    for line in csvreader:
            
        # Insert data into 'song_library' table
        session.execute(iquery1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        
        # Insert data into 'artist_library' table
        session.execute(iquery2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        
        # Insert data into 'playback_library' table
        session.execute(iquery3, (line[9], int(line[10]), line[1], line[4]))

#### Verify data has been inserted into each table

In [74]:
## VERIFY 'song_library' TABLE ##############################
#############################################################

query1 = "select * from song_library LIMIT 5"
data = []
column_labels = ('sessionid', 'iteminsession', 'artist', 'song_name', 'song_length')

try:
    rows = session.execute(query1)
    
except Exception as e:
    print(e)
    
for row in rows:
    #print (row.sessionid, row.iteminsession, row.artist, row.song_name, row.song_length)
    data.append([row.sessionid, row.iteminsession, row.artist, row.song_name, row.song_length])
    
query_df = pd.DataFrame(data, columns = column_labels)
query_df.head()

,sessionid,iteminsession,artist,song_name,song_length
0,23,0,Regina Spektor,The Calculation (Album Version),191.085266
1,23,1,Octopus Project,All Of The Champs That Ever Lived,250.957916
2,23,2,Tegan And Sara,So Jealous,180.061584
3,23,3,Dragonette,Okay Dolores,153.390564
4,23,4,Lil Wayne / Eminem,Drop The World,229.589752


In [75]:
## VERIFY 'artist_library' TABLE ##############################
#############################################################

query2 = "select * from artist_library LIMIT 5"
data = []
column_labels = ('userid', 'sessionid', 'iteminsession', 'artist', 'song_name', 'firstname', 'lastname')

try:
    rows = session.execute(query2)
    
except Exception as e:
    print(e)
    
for row in rows:
    #print (row.userid, row.sessionid, row.iteminsession, row.artist, row.song_name, row.firstname, row.lastname)
    data.append([row.userid, row.sessionid, row.iteminsession, row.artist, row.song_name, row.firstname, row.lastname])
    
query_df = pd.DataFrame(data, columns = column_labels)
query_df.head()

,userid,sessionid,iteminsession,artist,song_name,firstname,lastname
0,58,768,0,System of a Down,Sad Statue,Emily,Benson
1,58,768,1,Ghostland Observatory,Stranger Lover,Emily,Benson
2,58,768,2,Evergreen Terrace,Zero,Emily,Benson
3,85,776,2,Deftones,Head Up (LP Version),Kinsley,Young
4,85,776,3,The Notorious B.I.G.,Playa Hater (Amended Version),Kinsley,Young


In [76]:
## VERIFY 'artist_library' TABLE ##############################
#############################################################

query3 = "select * from playback_library LIMIT 5"
data = []
column_labels = ('userid', 'firstname', 'lastname', 'song_name')

try:
    rows = session.execute(query3)
    
except Exception as e:
    print(e)
    
for row in rows:
    #print (row.userid, row.firstname, row.lastname, row.song_name)
    data.append([row.userid, row.firstname, row.lastname, row.song_name])
    
query_df = pd.DataFrame(data, columns = column_labels)
query_df.head()

,userid,firstname,lastname,song_name
0,49,Chloe,Cuevas,Wonder What's Next
1,49,Chloe,Cuevas,In The Dragon's Den
2,44,Aleena,Kirby,Too Tough (1994 Digital Remaster)
3,49,Chloe,Cuevas,Rio De Janeiro Blue (Album Version)
4,15,Lily,Koch,My Place


## Query Each Table

##### Question 1: Need the artist, song title, and song's length in the music app history that was heard during {sessionId = 338}, and {itemInSession = 4}

In [77]:
qquery1 = "select artist, song_name, song_length from song_library WHERE sessionId = 338 AND itemInSession = 4"

data = []
column_labels = ('artist', 'song_name', 'song_length')

try:
    rows = session.execute(qquery1)
    
except Exception as e:
    print(e)
    
for row in rows:
    data.append([row.artist, row.song_name, row.song_length])
    
query_df = pd.DataFrame(data, columns = column_labels)
query_df.head()

,artist,song_name,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


##### Question 2: Need the artist, song, and user's first and last names for {userid = 10}, {sessionid = 182} and sorted by {itemInSession}

In [78]:
qquery2 = "select artist, song_name, firstname, lastname from artist_library WHERE userid = 10 AND sessionid = 182"

data = []
column_labels = ('artist', 'song_name', 'firstname', 'lastname')

try:
    rows = session.execute(qquery2)
    
except Exception as e:
    print(e)
    
for row in rows:
    data.append([row.artist, row.song_name, row.firstname, row.lastname])
    
query_df = pd.DataFrame(data, columns = column_labels)
query_df.head()

,artist,song_name,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


##### Question 3: Need all the first and last names who listened to the song {'All Hands Against His Own'}

In [79]:
qquery3 = "select userid, firstname, lastname from playback_library WHERE song_name = 'All Hands Against His Own'"

data = []
column_labels = ('firstname', 'lastname')

try:
    rows = session.execute(qquery3)
    
except Exception as e:
    print(e)

for row in rows:
    data.append([row.firstname, row.lastname])
    
query_df = pd.DataFrame(data, columns = column_labels)
query_df.head()

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [62]:
# Create DROP TABLE queries
dquery1 = "DROP TABLE song_library"
dquery2 = "DROP TABLE artist_library"
dquery3 = "DROP TABLE playback_library"

# Drop tables 
try:
    rows = session.execute(dquery1)
except Exception as e:
    print("Drop song_library: {}".format(e))
    
try:
    rows = session.execute(dquery2)
except Exception as e:
    print("Drop artist_library: {}".format(e))
    
try:
    rows = session.execute(dquery3)
except Exception as e:
    print("Drop playback_library: {}".format(e))

### Close the session and cluster connection¶

In [63]:
session.shutdown()
cluster.shutdown()